In [ ]:
!pip install --quiet datasets
!pip install --quiet frozendict
!pip install --quiet transformers
!pip install --quiet qatch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 26.9 MB/s eta 0:00:00


In [ ]:
import warnings
import os
import pandas as pd
from google.colab import drive
import pickle
import torch
import numpy as np
from transformers import TapasTokenizer


import collections
import numpy as np
from collections import defaultdict

from transformers import TapasConfig, TapasForQuestionAnswering
from torch.optim import AdamW

from torch.optim.lr_scheduler import ReduceLROnPlateau
from qatch import MetricEvaluator
from qatch.database_reader import MultipleDatabases
import logging
import warnings


warnings.filterwarnings("ignore", category=FutureWarning)

logging.basicConfig(level=logging.ERROR)  # Suppresses log messages below ERROR level
warnings.filterwarnings("ignore", "Table should be a list of list.*")  # Specifically ignores this warning


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title get_sql_tag
def get_sql_query_level(sql_query):
    # Function to classify SQL query and return just the level tag
    def classify_sql_query_level(sql_query):
        sql_query_lower = sql_query.lower()
        level = 6  # Default to OTHER

        if "select" in sql_query_lower and "from" in sql_query_lower:
            level = 0  # SELECT
            if "max(" in sql_query_lower or "min(" in sql_query_lower or \
               "avg(" in sql_query_lower or "count(" in sql_query_lower or \
               "sum(" in sql_query_lower:
                level = 1  # SIMPLE_AGGR

        if "where" in sql_query_lower:
            level = 2  # WHERE

        if "group by" in sql_query_lower:
            level = 3  # GROUPBY

        if "having" in sql_query_lower:
            level = 4  # HAVING

        if "order by" in sql_query_lower:
            level = 5  # ORDERBY

        return level

    # Classification tags
    classification_tags = {
        0: "SELECT",
        1: "SIMPLE_AGGR",
        2: "WHERE",
        3: "GROUPBY",
        4: "HAVING",
        5: "ORDERBY",
        6: "OTHER"
    }

    # Classify the SQL query
    level = classify_sql_query_level(sql_query)

    # Return the classification tag
    return classification_tags[level]

In [ ]:
db_save_path_spider = '/content/drive/MyDrive/spider/test_database'
databasesSpider = MultipleDatabases(db_save_path_spider)
evaluator_Spider = MetricEvaluator(databases=databasesSpider)

with open('/content/drive/MyDrive/spider/db_table_to_df_valid.pkl', 'rb') as file:
    db_to_df_spiderTest = pickle.load(file)

with open('/content/drive/MyDrive/spider/db_table_to_df.pkl', 'rb') as file:
    db_to_df_spiderTrain = pickle.load(file)

spider_train_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final.pkl')
spider_train_pd=spider_train_pd.drop(columns=["query_toks","query_toks_no_value","question_toks","operator"])
spider_train_pd['sql_tags'] = spider_train_pd['query'].apply(get_sql_query_level)

spider_test_pd = pd.read_pickle('/content/drive/MyDrive/spider/SPIDER_Simple_cleaned_final_valid.pkl')
spider_test_pd=spider_test_pd.drop(columns=["query_toks","query_toks_no_value","question_toks","operator"])
spider_test_pd['sql_tags'] = spider_test_pd['query'].apply(get_sql_query_level)

In [ ]:
##  train/valid split for Spider trainset
percentage_valid = 0.2
valid_indices = spider_train_pd.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_valid, random_state=42)).index
valid_df = spider_train_pd.loc[valid_indices]
train_df = spider_train_pd.drop(valid_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {spider_train_pd.shape[0]}")
print(f"Training set size: {train_df.shape[0]}")
print(f"Validation set size: {valid_df.shape[0]}")

Total dataset size: 1441
Training set size: 1154
Validation set size: 287


In [ ]:
train_df['sql_tags'].value_counts()

sql_tags
WHERE          441
ORDERBY        241
SELECT         203
SIMPLE_AGGR    178
HAVING          90
GROUPBY          1
Name: count, dtype: int64

In [ ]:
valid_df['sql_tags'].value_counts()

sql_tags
WHERE          110
ORDERBY         60
SELECT          51
SIMPLE_AGGR     44
HAVING          22
Name: count, dtype: int64

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
with open('/content/drive/MyDrive/ProprietaryDatasetsDict.pkl', 'rb') as f:
      db_to_df_proprietary = pickle.load(f)

db_save_path = '/content/drive/MyDrive/ProprietaryDatasets/database4'
databases = MultipleDatabases(db_save_path)
evaluator_Proprietary = MetricEvaluator(databases=databases)

qatch_pickle = pd.read_pickle('/content/drive/MyDrive/spider/QATCH_proprietary_v2.pkl')

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
ProprietaryDomains=['medicine','miscellaneous','ecommerce','finance']
qatch_pickle=qatch_pickle[qatch_pickle['domain']==ProprietaryDomains[0]]
qatch_pickle['sql_tags'] = qatch_pickle['sql_tags'].apply(lambda x: x.split('-')[0])## So that we can Having/select Having grouped together and not their subcategories

<ipython-input-8-e8883be1b539>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qatch_pickle['sql_tags'] = qatch_pickle['sql_tags'].apply(lambda x: x.split('-')[0])## So that we can Having/select Having grouped together and not their subcategories


In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
qatch_pickle['sql_tags'].unique()

array(['SELECT', 'ORDERBY', 'DISTINCT', 'WHERE', 'GROUPBY', 'HAVING',
       'SIMPLE'], dtype=object)

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
percentage_valid = 0.2
temp_indices = qatch_pickle.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_valid, random_state=42)).index
temp_df = qatch_pickle.loc[temp_indices]
train_df_full = qatch_pickle.drop(temp_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {qatch_pickle.shape[0]}")
print(f"Training set size: {train_df_full.shape[0]}")
print(f"Tempset size: {temp_df.shape[0]}")

Total dataset size: 429
Training set size: 344
Tempset size: 85


In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
percentage_test = 0.5
test_indices = temp_df.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_test, random_state=42)).index
test_df = qatch_pickle.loc[test_indices]
valid_df = temp_df.drop(test_indices)

# Check the size of each set to confirm the split
print(f"Total dataset size: {temp_df.shape[0]}")
print(f"Training set size: {valid_df.shape[0]}")
print(f"Tempset size: {test_df.shape[0]}")

Total dataset size: 85
Training set size: 42
Tempset size: 43


In [ ]:
# only execute when you want to test/fine tune the model on proprieatry data

# Now lets sample from the trainset as we are not going to use all of it
# At this stage I am still waiting for Simone to change qatch so that I can control
# the amount of queries in each category, and increase the amout of data
# percentage_train = 0.8
# train_indices = train_df_full.groupby('sql_tags', group_keys=False).apply(lambda x: x.sample(frac=percentage_train, random_state=42)).index
# train_df = train_df_full.loc[train_indices]

# # Check the size of each set to confirm the split
# print(f"Total dataset size: {train_df_full.shape[0]}")
# print(f"Training set size: {train_df.shape[0]}")

train_df = train_df_full.copy()

In [ ]:
#@title stratification  of qatch Legacy


# percentage = 1
# number_of_example = int(percentage * train_df_full.shape[0])

# # Define a function to safely sample or take all if the sample size is too large
# def safe_sample(df, n):
#     return df if len(df) < n else df.sample(n=n)

# qatch_pickle_DISTINCT = safe_sample(train_df_full[train_df_full['sql_tags'] == 'DISTINCT'], int(0.1 * number_of_example))
# qatch_pickle_GROUPBY = safe_sample(train_df_full[train_df_full['sql_tags'] == 'GROUPBY'], int(0.1 * number_of_example))
# qatch_pickle_HAVING = safe_sample(train_df_full[train_df_full['sql_tags'] == 'HAVING'], int(0.20 * number_of_example))
# qatch_pickle_ORDERBY = safe_sample(train_df_full[train_df_full['sql_tags'] == 'ORDERBY'], int(0.15 * number_of_example))
# qatch_pickle_SELECT = safe_sample(train_df_full[train_df_full['sql_tags'] == 'SELECT'], int(0.15 * number_of_example))
# qatch_pickle_SIMPLE = safe_sample(train_df_full[train_df_full['sql_tags'] == 'SIMPLE'], int(0.25 * number_of_example))
# qatch_pickle_WHERE = safe_sample(train_df_full[train_df_full['sql_tags'] == 'WHERE'], int(0.15 * number_of_example))

# train_df = pd.concat([qatch_pickle_SIMPLE, qatch_pickle_WHERE, qatch_pickle_SELECT, qatch_pickle_HAVING, qatch_pickle_ORDERBY, qatch_pickle_DISTINCT, qatch_pickle_GROUPBY])


# # Check the size of each set to confirm the split
# print(f"Total dataset size: {train_df_full.shape[0]}")
# print(f"Training set size: {train_df.shape[0]}")
# # train_df = pd.concat([train_df, sampled_df], axis=0)

In [ ]:
#only execute when you want to test/fine tune the model on proprieatry data
train_df['sql_tags'].value_counts()

sql_tags
HAVING      149
WHERE        77
SELECT       32
ORDERBY      26
SIMPLE       25
GROUPBY      22
DISTINCT     13
Name: count, dtype: int64

In [ ]:
# initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

In [ ]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer,db_to_df):
        self.df = df
        self.db_to_df=db_to_df
        self.tokenizer = tokenizer
        self.problematic_ids = []  # Initialize a list to store IDs of problematic questions

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        table_name = item['table_used']## for prorietary data
        # table_name = item['seq_id']
        try:
            table = self.db_to_df[table_name].astype(str)
            encoding = self.tokenizer(
                table=table,
                queries=item.question,
                answer_coordinates=item.answer_coordinates,
                answer_text=item.answer_text,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            float_answer = torch.tensor(item.float_value)
            encoding = {key: val.squeeze(0) for key, val in encoding.items()}
            encoding["float_answer"] = float_answer


        except Exception as e:
            print(f"Error processing index {idx} (ID: {item['ID']}): {e}")
            self.problematic_ids.append(item.ID)  # Store the ID of the problematic question
            encoding = None

        return encoding


    def __len__(self):
        return len(self.df)

    def get_problematic_ids(self):
        # Returns the list of problematic IDs
        return self.problematic_ids

# Custom collate function to filter out None values, if not already implemented
def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]  # Filter out None values
    if not batch:
        return None
    return torch.utils.data.dataloader.default_collate(batch)

In [ ]:
#@title qatch data
# qatch_pickle = pd.read_pickle('/content/drive/MyDrive/spider/QATCH_SPIDER.pkl')


# percentage = 1
# number_of_example = int(percentage * train_df.shape[0])


# # qatch_pickle_SIMPLE_AGG = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SIMPLE-AGG-')].sample(n=int(0.3 * number_of_example))
# # qatch_pickle_WHERE = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^WHERE-')].sample(n=int(0.15 * number_of_example))
# # qatch_pickle_SELECT = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SELECT-')].sample(n=int(0.30 * number_of_example))
# # qatch_pickle_HAVING = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^HAVING-')].sample(n=int(0.15 * number_of_example))
# # qatch_pickle_ORDERBY = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^ORDERBY-')].sample(n=int(0.1 * number_of_example))

# # qatch_pickle_SIMPLE_AGG = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SIMPLE-AGG-')].sample(n=int(0.25 * number_of_example))
# # qatch_pickle_WHERE = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^WHERE-')].sample(n=int(0.1 * number_of_example))
# # qatch_pickle_SELECT = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SELECT-')].sample(n=int(0.30 * number_of_example))
# # qatch_pickle_HAVING = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^HAVING-')].sample(n=int(0.25 * number_of_example))
# # qatch_pickle_ORDERBY = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^ORDERBY-')].sample(n=int(0.1 * number_of_example))

# # qatch_pickle_SIMPLE_AGG = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SIMPLE-AGG-')].sample(n=int(0.25 * number_of_example))
# # qatch_pickle_WHERE = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^WHERE-')].sample(n=int(0.15 * number_of_example))
# # qatch_pickle_SELECT = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SELECT-')].sample(n=int(0.25 * number_of_example))
# # qatch_pickle_HAVING = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^HAVING-')].sample(n=int(0.25 * number_of_example))
# # qatch_pickle_ORDERBY = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^ORDERBY-')].sample(n=int(0.1 * number_of_example))


# qatch_pickle_SIMPLE_AGG = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SIMPLE-AGG-')].sample(n=int(0.3 * number_of_example))
# qatch_pickle_WHERE = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^WHERE-')].sample(n=int(0.15 * number_of_example))
# qatch_pickle_SELECT = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^SELECT-')].sample(n=int(0.20 * number_of_example))
# qatch_pickle_HAVING = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^HAVING-')].sample(n=int(0.30 * number_of_example))
# qatch_pickle_ORDERBY = qatch_pickle[qatch_pickle['sql_tags'].str.match(r'^ORDERBY-')].sample(n=int(0.1 * number_of_example))

# sampled_df = pd.concat([qatch_pickle_SIMPLE_AGG, qatch_pickle_WHERE, qatch_pickle_SELECT, qatch_pickle_HAVING, qatch_pickle_ORDERBY])
# train_df = pd.concat([train_df, sampled_df], axis=0)


# train_df = train_df.drop(columns=["ID"], errors='ignore')
# train_df.reset_index(inplace=True)
# train_df.rename(columns={'index': 'ID'}, inplace=True)


In [ ]:
train_df_copy=train_df.copy()
train_dataset = TableDataset(df=train_df_copy, tokenizer=tokenizer,db_to_df=db_to_df_proprietary)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=custom_collate_fn)

for idx, batch in enumerate(train_dataloader):
    if batch is not None:
        continue
    else:
        print(f"Batch {idx} skipped due to problematic data")

print(train_dataset.get_problematic_ids())


prob=train_dataset.get_problematic_ids()
train_df_copy = train_df_copy[~train_df_copy['ID'].isin(prob)]
train_dataset = TableDataset(df=train_df_copy, tokenizer=tokenizer,db_to_df=db_to_df_proprietary)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=custom_collate_fn)

[]


In [ ]:
with open('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/DataLoaders/train_dataloader_spider.pkl', 'wb') as f:
    pickle.dump(train_dataloader, f)
# with open('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/DataLoaders/train_dataloader_spider.pkl', 'rb') as file:
#     train_dataloader = pickle.load(file)

In [ ]:
#Number of batches of ==> 144 batches
print(len(train_dataloader))

35


In [ ]:
#@title Computing predictions Proprietary
def getAnswerProprietary(row):
    row_index = row['ID']
    table_used=row['table_used']
    table = db_to_df_proprietary[table_used]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        model_input = row['input'].to('cuda')
        # Step 1: Pass the model_input to the model for forward propagation to generate outputs.
        outputs = model(**model_input)
        # Step 2: Move the model_input tensor to cpu.
        model_input.to('cpu')
        # Step 3: Move the output tensors to CPU and detach them from the computational graph.
        outputs = {idx: outputs[idx].cpu().detach() for idx in outputs}
        # Step 4: Detach any tensors in model_input from the computational graph.
        [model_input[idx].detach() for idx in model_input]

        # Step 5: Retrieve the coordinates from the logits using a tokenizer function.
        pred_query_cords, _ = tokenizer.convert_logits_to_predictions(
            model_input,
            outputs['logits'],
            outputs['logits_aggregation']
        )
        answers = []
        for tbl_cords in pred_query_cords:
            query_answer = defaultdict(list)
            # For each coordinate set in the predicted query coordinates,
            # construct a dictionary of row-wise query answers.
            [query_answer[row].append(table.iat[(row, col)])
             for row, col in tbl_cords]
            answers.extend(list(query_answer.values()))
        # print(f"Row {row_index} processed successfully.")
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return answers

def getInputProprietary(row):
    row_index = row['ID']

    table_used=row['table_used']
    table = db_to_df_proprietary[table_used]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        inputs = tokenizer(table=table, queries=query_nl, padding='max_length', return_tensors="pt", truncation=True)
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return inputs

In [ ]:
#@title Computing prediction Spider train and valid
def getAnswerSpider(row):
    row_index = row['ID']
    seq_id = row["db_id"] + "_X_" + row["table_used"]
    if seq_id not in db_to_df_spiderTrain or db_to_df_spiderTrain[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None
    table=db_to_df_spiderTrain[seq_id]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        model_input = row['input'].to('cuda')
        # Step 1: Pass the model_input to the model for forward propagation to generate outputs.
        outputs = model(**model_input)
        # Step 2: Move the model_input tensor to cpu.
        model_input.to('cpu')
        # Step 3: Move the output tensors to CPU and detach them from the computational graph.
        outputs = {idx: outputs[idx].cpu().detach() for idx in outputs}
        # Step 4: Detach any tensors in model_input from the computational graph.
        [model_input[idx].detach() for idx in model_input]

        # Step 5: Retrieve the coordinates from the logits using a tokenizer function.
        pred_query_cords, _ = tokenizer.convert_logits_to_predictions(
            model_input,
            outputs['logits'],
            outputs['logits_aggregation']
        )
        answers = []
        for tbl_cords in pred_query_cords:
            query_answer = defaultdict(list)
            # For each coordinate set in the predicted query coordinates,
            # construct a dictionary of row-wise query answers.
            [query_answer[row].append(table.iat[(row, col)])
             for row, col in tbl_cords]
            answers.extend(list(query_answer.values()))
        # print(f"Row {row_index} processed successfully.")
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return answers

def getInputSpider(row):
    row_index = row['ID']
    seq_id = row["db_id"] + "_X_" + row["table_used"]
    if seq_id not in db_to_df_spiderTrain or db_to_df_spiderTrain[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None
    table=db_to_df_spiderTrain[seq_id]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        inputs = tokenizer(table=table, queries=query_nl, padding='max_length', return_tensors="pt", truncation=True)
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return inputs

In [ ]:
#@title Computing prediction Spider Test/Dev
def getAnswerSpiderTest(row):
    row_index = row['ID']
    seq_id = row["db_id"] + "_X_" + row["table_used"]
    if seq_id not in db_to_df_spiderTest or db_to_df_spiderTest[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None
    table=db_to_df_spiderTest[seq_id]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        model_input = row['input'].to('cuda')
        # Step 1: Pass the model_input to the model for forward propagation to generate outputs.
        outputs = model(**model_input)
        # Step 2: Move the model_input tensor to cpu.
        model_input.to('cpu')
        # Step 3: Move the output tensors to CPU and detach them from the computational graph.
        outputs = {idx: outputs[idx].cpu().detach() for idx in outputs}
        # Step 4: Detach any tensors in model_input from the computational graph.
        [model_input[idx].detach() for idx in model_input]

        # Step 5: Retrieve the coordinates from the logits using a tokenizer function.
        pred_query_cords, _ = tokenizer.convert_logits_to_predictions(
            model_input,
            outputs['logits'],
            outputs['logits_aggregation']
        )
        answers = []
        for tbl_cords in pred_query_cords:
            query_answer = defaultdict(list)
            # For each coordinate set in the predicted query coordinates,
            # construct a dictionary of row-wise query answers.
            [query_answer[row].append(table.iat[(row, col)])
             for row, col in tbl_cords]
            answers.extend(list(query_answer.values()))
        # print(f"Row {row_index} processed successfully.")
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return answers

def getInputSpiderTest(row):
    row_index = row['ID']
    seq_id = row["db_id"] + "_X_" + row["table_used"]
    if seq_id not in db_to_df_spiderTest or db_to_df_spiderTest[seq_id].empty:
        print(f"Row {row_index} skipped: table not found or empty.")
        return None
    table=db_to_df_spiderTest[seq_id]
    table = table.astype(str)
    query_nl = [row["question"]]

    try:
        inputs = tokenizer(table=table, queries=query_nl, padding='max_length', return_tensors="pt", truncation=True)
    except IndexError:
        print(f"Row {row_index} skipped due to IndexError.")
        return None

    return inputs

In [ ]:
valid_dataset = TableDataset(df=valid_df, tokenizer=tokenizer,db_to_df=db_to_df_proprietary)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=8, collate_fn=custom_collate_fn)

for idx, batch in enumerate(valid_dataloader):
    if batch is not None:
        continue
    else:
        print(f"Batch {idx} skipped due to problematic data")

print(valid_dataset.get_problematic_ids())


prob_valid=train_dataset.get_problematic_ids()
valid_df = valid_df[~valid_df['ID'].isin(prob_valid)]
valid_dataset = TableDataset(df=valid_df, tokenizer=tokenizer,db_to_df=db_to_df_proprietary)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=8, collate_fn=custom_collate_fn)

[]


In [ ]:
with open('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/DataLoaders/valid_dataloader_spider.pkl', 'wb') as f:
    pickle.dump(valid_dataloader, f)
# with open('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/DataLoaders/valid_dataloader_spider.pkl', 'rb') as file:
#     valid_dataloader = pickle.load(file)

In [ ]:
print(len(valid_dataloader))

6


In [ ]:
valid_df['input'] = valid_df.apply(getInputProprietary, axis=1)

In [ ]:
test_df['input'] = test_df.apply(getInputProprietary, axis=1)

In [ ]:
# this is the default WTQ configuration
config = TapasConfig(
    num_aggregation_labels=4,# MAX MIN COUNT AVG
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
)
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)
optimizer = AdamW(model.parameters(), lr=5e-5)
model.to('cuda')

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['aggregation_classifier.bias', 'aggregation_classifier.weight', 'column_output_bias', 'column_output_weights', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0-11): 12 x TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 110676486


In [ ]:
#@title Freeing  the gpu
model_object = model.cuda()

del model_object  #deleting the model


# model will still be on cache until its place is taken by other objects so also execute the below lines
import gc         # garbage collect library
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#@title calculate_and_log_stats
def calculate_and_log_stats(epoch,avg_loss,df,SpiderOrProprietary):
    #SpiderOrProprietary=0 for spidertrain and valid, 1 for spidertest, 2 for Proprietary
    getAnswer,evaluator=None,None
    if SpiderOrProprietary==0:
      getAnswer=getAnswerSpider
      evaluator=evaluator_Spider
    elif SpiderOrProprietary==1:
      getAnswer=getAnswerSpiderTest
      evaluator=evaluator_Spider
    elif SpiderOrProprietary==2:
      getAnswer=getAnswerProprietary
      evaluator=evaluator_Proprietary
    else:
      return "SpiderOrProprietary=0 for spidertrain and valid, 1 for spidertest, 2 for Proprietary"

    model.eval()
    with torch.no_grad():
      df_cp=df.copy()
      df_cp['predictions_TAPAS'] = df_cp.apply(getAnswer, axis=1)
      tests_df_results = evaluator.evaluate_with_df(df_cp,
                                            prediction_col_name=f'predictions_TAPAS',
                                            task="QA")
    model.train()

    columns_to_average = [
        'cell_precision_predictions_TAPAS',
        'cell_recall_predictions_TAPAS',
        'tuple_cardinality_predictions_TAPAS',
        'tuple_constraint_predictions_TAPAS',
        'tuple_order_predictions_TAPAS'
    ]


    general_stats = {
        'epoch': epoch,
        'avg_loss': avg_loss,
        'general_avg_cell_precision': tests_df_results[columns_to_average[0]].mean(),
        'general_avg_cell_recall': tests_df_results[columns_to_average[1]].mean(),
        'general_avg_tuple_cardinality': tests_df_results[columns_to_average[2]].mean(),
        'general_avg_tuple_constraint': tests_df_results[columns_to_average[3]].mean()
    }

    print(f"General Model Performance Metrics for epoch:{epoch}")
    for stat_name, stat_value in general_stats.items():
        if stat_name != 'epoch':  # We don't need to print the epoch again
            print(f"{stat_name}: {stat_value}")
    print("----------------------------")

    grouped_means = tests_df_results.groupby('sql_tags')[columns_to_average].mean()


    for sql_tag, row in grouped_means.iterrows():
        for col in columns_to_average:
            key_name = f"{sql_tag}_{col}"
            general_stats[key_name] = row[col]

    print("Grouped Stats by SQL Tag:")
    for sql_tag, metrics in grouped_means.iterrows():
        print(f"SQL Tag: {sql_tag}")
        for col in columns_to_average:
            print(f"  {col}: {metrics[col]}")
        print("----------------------------")
    return general_stats


In [ ]:
#@title estimate_valid_loss
def estimate_valid_loss(dataloader):
    model.eval()
    with torch.no_grad():
        val_loss=0
        for batch in dataloader:
          input_ids = batch["input_ids"].to('cuda')
          attention_mask = batch["attention_mask"].to('cuda')
          token_type_ids = batch["token_type_ids"].to('cuda')
          labels = batch["labels"].to('cuda')
          numeric_values = batch["numeric_values"].to('cuda')
          numeric_values_scale = batch["numeric_values_scale"].to('cuda')
          float_answer = batch["float_answer"].to('cuda')

          outputs = model(
              input_ids=input_ids,
              attention_mask=attention_mask,
              token_type_ids=token_type_ids,
              labels=labels,
              numeric_values=numeric_values,
              numeric_values_scale=numeric_values_scale,
              float_answer=float_answer,
          )
          loss = outputs.loss
          val_loss+=loss.item()
    model.train()
    return val_loss/len(dataloader)


In [ ]:
#@title Loading an existing model
#Remember to save the optimizer only if we want to resume the training of the
#model, if the model is going to be used for inefrence only, there is no need
#Only the state is sufficient
# save_path = '/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/'
# checkpoint_path = os.path.join(save_path, "SpiderCheckpointProprietary60prct.pth")

save_path = '/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/'
checkpoint_path = os.path.join(save_path, "SpiderCheckpoint.pth")
if os.path.exists(checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)#,map_location=torch.device('cpu')

    # # Restore the model and optimizer states
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print("Checkpoint loaded successfully.")
else:
    print(f"No checkpoint found at {checkpoint_path}")

Checkpoint loaded successfully.


In [ ]:
#@title Traning loop

stats_columns = ['epoch', 'avg_loss']
stats_df = pd.DataFrame(columns=stats_columns)
loss_logs_df_columns = ['epoch', 'batch_count', 'valid_loss', 'interval_avg_loss']
loss_logs_df=pd.DataFrame(columns=loss_logs_df_columns)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
eval_iters=7
best_valid_loss = float('inf')
early_stopping=6

count_stop=0
BestModel_checkpoint_path='/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/SpiderCheckpointProprietary80prct.pth'

for epoch in range(25):
    print("Epoch:", epoch)
    epoch_loss = 0.0
    batch_count = 0
    batch_count_loss = 0
    epoch_loss_logs = []


    for batch in train_dataloader:
        batch_count+=1
        input_ids = batch["input_ids"].to('cuda')
        attention_mask = batch["attention_mask"].to('cuda')
        token_type_ids = batch["token_type_ids"].to('cuda')
        labels = batch["labels"].to('cuda')
        numeric_values = batch["numeric_values"].to('cuda')
        numeric_values_scale = batch["numeric_values_scale"].to('cuda')
        float_answer = batch["float_answer"].to('cuda')

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=float_answer,
        )
        loss = outputs.loss
        epoch_loss += loss.item()
        batch_count_loss+=loss.item()

        loss.backward()
        optimizer.step()
        if batch_count % eval_iters == 0 or batch_count == len(train_dataloader):
            valid_loss = estimate_valid_loss(valid_dataloader)
            interval_batches = eval_iters if batch_count % eval_iters == 0 else batch_count % eval_iters
            interval_avg_loss = batch_count_loss / interval_batches
            print(f"epoch: {epoch}, batch count: {batch_count}, valid_loss: {valid_loss}, interval_avg_loss: {interval_avg_loss}")
            batch_count_loss = 0

            epoch_loss_logs.append({'epoch': epoch, 'batch_count': batch_count, 'valid_loss': valid_loss, 'interval_avg_loss': interval_avg_loss})
    if epoch_loss_logs:
        loss_logs_df = pd.concat([loss_logs_df, pd.DataFrame(epoch_loss_logs)], ignore_index=True)

    scheduler.step(valid_loss)
    if  valid_loss < best_valid_loss and abs(valid_loss-best_valid_loss)>0.001:
        best_valid_loss = valid_loss
        torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        },BestModel_checkpoint_path)
        print(f"Epoch {epoch}: New best validation loss {valid_loss}, model saved.")
        count_stop=0
    else:
      count_stop+=1
    if count_stop>early_stopping:
      break

    avg_loss=epoch_loss /len(train_dataloader)
loss_logs_df.to_csv('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/SpiderResults/Tapas_Spider_loss_Proprietary80prct.csv', index=False)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch: 0
epoch: 0, batch count: 7, valid_loss: 1.3543595671653748, interval_avg_loss: 1.1537216092858995
epoch: 0, batch count: 14, valid_loss: 1.632951060930888, interval_avg_loss: 0.2996663344757898
epoch: 0, batch count: 21, valid_loss: 1.4849501798550289, interval_avg_loss: 1.5277460579361235
epoch: 0, batch count: 28, valid_loss: 2.103793611129125, interval_avg_loss: 2.558841756411961
epoch: 0, batch count: 35, valid_loss: 1.579394777615865, interval_avg_loss: 2.3063539947782243
Epoch 0: New best validation loss 1.579394777615865, model saved.
Epoch: 1
epoch: 1, batch count: 7, valid_loss: 1.4387971262137096, interval_avg_loss: 1.5387390000479562
epoch: 1, batch count: 14, valid_loss: 1.4843517591555913, interval_avg_loss: 0.6983676680496761
epoch: 1, batch count: 21, valid_loss: 0.9072960217793783, interval_avg_loss: 1.3727972890649522
epoch: 1, batch count: 28, valid_loss: 1.3250787556171417, interval_avg_loss: 2.511566093989781
epoch: 1, batch count: 35, valid_loss: 1.993857155

In [ ]:
#@title Saving the model
# save_path = '/content/drive/MyDrive/model_checkpoints'

# # Save the model and optimizer state at the end of training
# if not os.path.exists(save_path):
#     os.makedirs(save_path)
# checkpoint_path = os.path.join(save_path, "model_spiderMine100prct_qatch.pt")
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
# }, checkpoint_path)
# print(f"Final checkpoint saved to {checkpoint_path}")

In [ ]:
valid_df_stats=calculate_and_log_stats(None,None,valid_df,SpiderOrProprietary=2)

Evaluating QA tests: 100%|██████████| 42/42 [00:01<00:00, 38.89it/s]


General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.6460476190476191
general_avg_cell_recall: 0.4214047619047621
general_avg_tuple_cardinality: 0.6971190476190477
general_avg_tuple_constraint: 0.3333333333333333
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: DISTINCT
  cell_precision_predictions_TAPAS: 1.0
  cell_recall_predictions_TAPAS: 1.0
  tuple_cardinality_predictions_TAPAS: 1.0
  tuple_constraint_predictions_TAPAS: 1.0
  tuple_order_predictions_TAPAS: nan
----------------------------
SQL Tag: GROUPBY
  cell_precision_predictions_TAPAS: 1.0
  cell_recall_predictions_TAPAS: 0.6666666666666666
  tuple_cardinality_predictions_TAPAS: 1.0
  tuple_constraint_predictions_TAPAS: 0.3333333333333333
  tuple_order_predictions_TAPAS: nan
----------------------------
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.34210526315789475
  cell_recall_predictions_TAPAS: 0.47368421052631576
  tuple_cardinality_predictions_TAPAS: 0.5
  

In [ ]:
valid_stats_df = pd.DataFrame([valid_df_stats])
valid_stats_df.to_csv('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/SpiderResults/Tapas_qatchMetrics_medicine80prct_valid.csv', index=False)

In [ ]:
test_df_stats=calculate_and_log_stats(None,None,test_df,SpiderOrProprietary=2)

Evaluating QA tests: 100%|██████████| 43/43 [00:00<00:00, 119.00it/s]

General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.7343023255813953
general_avg_cell_recall: 0.481651162790698
general_avg_tuple_cardinality: 0.7187906976744186
general_avg_tuple_constraint: 0.38953488372093026
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: DISTINCT
  cell_precision_predictions_TAPAS: 1.0
  cell_recall_predictions_TAPAS: 1.0
  tuple_cardinality_predictions_TAPAS: 0.7855
  tuple_constraint_predictions_TAPAS: 0.625
  tuple_order_predictions_TAPAS: nan
----------------------------
SQL Tag: GROUPBY
  cell_precision_predictions_TAPAS: 1.0
  cell_recall_predictions_TAPAS: 0.5
  tuple_cardinality_predictions_TAPAS: 1.0
  tuple_constraint_predictions_TAPAS: 0.0
  tuple_order_predictions_TAPAS: nan
----------------------------
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.4351666666666667
  cell_recall_predictions_TAPAS: 0.5833333333333334
  tuple_cardinality_predictions_TAPAS: 0.6203888888888889
  tuple_constr

In [ ]:
test_stats_df = pd.DataFrame([test_df_stats])
test_stats_df.to_csv('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/SpiderResults/Tapas_qatchMetrics_medicine80prct_Test.csv', index=False)

In [ ]:
spider_test_pd['input'] = spider_test_pd.apply(getInputSpiderTest, axis=1)

In [ ]:
spider_test_stats=calculate_and_log_stats(None,None,spider_test_pd,SpiderOrProprietary=1)

Evaluating QA tests: 100%|██████████| 211/211 [00:10<00:00, 20.49it/s]

General Model Performance Metrics for epoch:None
avg_loss: None
general_avg_cell_precision: 0.4795829383886257
general_avg_cell_recall: 0.45483412322274885
general_avg_tuple_cardinality: 0.5076777251184835
general_avg_tuple_constraint: 0.3354881516587678
----------------------------
Grouped Stats by SQL Tag:
SQL Tag: HAVING
  cell_precision_predictions_TAPAS: 0.4965
  cell_recall_predictions_TAPAS: 0.7545833333333333
  tuple_cardinality_predictions_TAPAS: 0.44316666666666665
  tuple_constraint_predictions_TAPAS: 0.7545833333333333
  tuple_order_predictions_TAPAS: nan
----------------------------
SQL Tag: ORDERBY
  cell_precision_predictions_TAPAS: 0.9750000000000001
  cell_recall_predictions_TAPAS: 0.8286666666666667
  tuple_cardinality_predictions_TAPAS: 0.9339722222222223
  tuple_constraint_predictions_TAPAS: 0.6759444444444446
  tuple_order_predictions_TAPAS: 0.4584444444444445
----------------------------
SQL Tag: SELECT
  cell_precision_predictions_TAPAS: 1.0
  cell_recall_predict

In [ ]:
spider_stats_df = pd.DataFrame([spider_test_stats])
spider_stats_df.to_csv('/content/drive/MyDrive/TapasSpider_Propieratry_ASAP/SpiderResults/Spider_test_QatchMetrics_Tapas80prct.csv', index=False)